In [1]:
import numpy as np
import awkward as ak
import glob
import dask_awkward as dak
from distributed import LocalCluster, Client, progress

client =  Client(n_workers=60,  threads_per_worker=1, processes=True, memory_limit='8 GiB') 


In [14]:
# year = "2017"
year = "2016postVFP"
# load_path = "/depot/cms/users/yun79/hmm/copperheadV1clean/test_test//stage1_output/2018/f1_0/cutflow_data_*"
load_path = f"/depot/cms/users/yun79/hmm/copperheadV1clean/cutflow_18Apr2025_singleMuTrigMatch//stage1_output/{year}/f1_0/cutflow_*"
filelist = glob.glob(f"{load_path}/*.parquet")
ak_zip=dak.from_parquet(filelist)

In [15]:
ak_zip.fields

['TotalEntries',
 'LumiMaskMetFilterPv',
 'HLT_filter',
 'muon_base_selection',
 'muon_trig_match',
 'electron_veto',
 'Jet_selection_njetsLeq2',
 'HemVeto_filter',
 'anti_ttH_btag_cut',
 'ggH_cut',
 'signal_fit_region']

In [16]:
# # sig_region_yield = [ak_zip[field][:50] for field in ak_zip.fields]
# sig_region_yield = [ak_zip[field] for field in ak_zip.fields]
# # sig_region_yield = ak.prod(sig_region_yield)

In [17]:
sig_region_arr = ak.ones_like(ak_zip.TotalEntries)

for field in ak_zip.fields:
    # if "HemVeto" in field:
        # continue
    arr = ak_zip[field]
    sig_region_arr = sig_region_arr & arr
    print(f"{field} sum: {ak.sum(sig_region_arr).compute()}")

# sig_region_arr = ak.num(sig_region_arr, axis=0).compute()
sig_region_arr = ak.sum(sig_region_arr).compute()

TotalEntries sum: 331976208
LumiMaskMetFilterPv sum: 326911885
HLT_filter sum: 216656712
muon_base_selection sum: 12873562
muon_trig_match sum: 12361998
electron_veto sum: 12358288
Jet_selection_njetsLeq2 sum: 11890536
HemVeto_filter sum: 11890536
anti_ttH_btag_cut sum: 11780450
ggH_cut sum: 11719595
signal_fit_region sum: 188430


In [2]:
# Rereco

year = "2018"
# year = "*"
# year = "2016postVFP"
# load_path = "/depot/cms/users/yun79/hmm/copperheadV1clean/test_test//stage1_output/2018/f1_0/cutflow_data_*"
# load_path = f"/depot/cms/users/yun79/hmm/copperheadV1clean/cutflow_rereco_yun_Apr20//stage1_output/{year}/data_B"
load_path = f"/depot/cms/users/yun79/hmm/copperheadV1clean/cutflow_rereco_yun_Apr22/stage1_output/{year}/data_*"
filelist = glob.glob(f"{load_path}/*.parquet")
ak_zip=dak.from_parquet(filelist)

In [3]:
ak_zip.fields

['TotalEntries',
 'LumiMaskMetFilterPv',
 'HLT_filter',
 'muon_base_selection',
 'muon_trig_match',
 'electron_veto',
 'Jet_selection_njetsLeq2',
 'anti_ttH_btag_cut',
 'ggH_cut',
 'signal_fit_region',
 'njets_nominal',
 'jet1_pt_nominal',
 'jj_mass_nominal',
 'jj_dEta_nominal',
 'nBtagLoose_nominal',
 'nBtagMedium_nominal',
 'dataset',
 'year']

In [ ]:
sig_region_arr = ak.ones_like(ak_zip.TotalEntries)

fields2compute = [
    'TotalEntries',
 'LumiMaskMetFilterPv',
 'HLT_filter',
 'muon_base_selection',
 'muon_trig_match',
 'electron_veto',
 'Jet_selection_njetsLeq2',
 'anti_ttH_btag_cut',
 'ggH_cut',
 'signal_fit_region',
]

for field in ak_zip.fields:
    #
        # continue
    if field not in fields2compute:
        continue
    arr = ak_zip[field]
    # print(f"{field} arr: {arr.compute()}")
    if "Jet_selection_njetsLeq2" in field:
        sig_region_arr_copy = sig_region_arr & arr # we don't include njets <= 2 in our continuous selection
        # print(f"{field} sum: {ak.sum(sig_region_arr_copy).compute()}")
        
    else:
        sig_region_arr = sig_region_arr & arr
        # print(f"{field} sum: {ak.sum(sig_region_arr).compute()}")
    
print(f"{field} sum: {ak.sum(sig_region_arr).compute()}")

# sig_region_arr = ak.num(sig_region_arr, axis=0).compute()
# sig_region_arr = ak.sum(sig_region_arr).compute()

In [60]:
ak_zip["jet1_pt_nominal"][sig_region_arr].compute()
# ak_zip["njets_nominal"][sig_region_arr].compute()
# ak.max(ak_zip["jet1_pt_nominal"][sig_region_arr]).compute()

<Array [35.3, 25.6, 0, 40.4, 0, 32.5, 41.2, 0, 0, 0] type='10 * ?float32'>

In [62]:
# print(ak_zip["ggH_cut"][sig_region_arr].compute())
print(ak_zip["jet1_pt_nominal"][sig_region_arr].compute())

print(ak_zip["jj_mass_nominal"][sig_region_arr].compute())
print(ak_zip["jj_dEta_nominal"][sig_region_arr].compute())
print(ak_zip["nBtagLoose_nominal"][sig_region_arr].compute())
print(ak_zip["nBtagMedium_nominal"][sig_region_arr].compute())
print(ak_zip["njets_nominal"][sig_region_arr].compute())


[35.3, 25.6, 0, 40.4, 0, 32.5, 41.2, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 1, 0, 1, 0, 1, 1, 0, 0, 0]


In [25]:
ak_zip["njets_nominal"][sig_region_arr&ak_zip.Jet_selection_njetsLeq2].compute()


<Array [2, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0] type='11 * ?float64'>

In [5]:
ak.sum(ak_zip["signal_fit_region"]).compute()

np.int64(102765)

In [88]:
# find the root files with the run and lumi of interest
import glob
load_path = "/eos/purdue/store/group/local/hmm/nanoAODv6_private/FSRmyNanoProdData2017_NANOV4/SingleMuon/RunIISummer16MiniAODv3_FSRmyNanoProdData2017_NANOV4_un2017B-31Mar2018-v1/*/*"
# load_path = "/eos/purdue//store/data/Run2017B/SingleMuon/NANOAOD/02Apr2020-v1/*"
filelist =glob.glob(f"{load_path}/*.root")

In [89]:
# filelist

In [90]:
import concurrent.futures
import uproot
import awkward as ak

def getGoodFile(fname, run, luminosityBlock):
    up_file = uproot.open(fname) 
    run_cond = up_file["Events"]['run'].array() == run
    run_cond = ak.sum(run_cond) > 0
    luminosityBlock_cond = up_file["Events"]['luminosityBlock'].array() ==luminosityBlock
    luminosityBlock_cond = ak.sum(luminosityBlock_cond) > 0
    if run_cond and luminosityBlock_cond:
        return fname # if no problem, return empty string
    else:
        return "" # bad file


def getGoodFileParallelize(filelist, run, luminosityBlock, max_workers=60):
    file_l_len = len(filelist)
    runs = [run for _ in range(file_l_len)]
    luminosityBlocks = [luminosityBlock for _ in range(file_l_len)]
    with concurrent.futures.ProcessPoolExecutor(max_workers=max_workers) as executor:
        # Submit each file check to the executor
        results = list(executor.map(getGoodFile, filelist, runs, luminosityBlocks))
    
    good_file_l = []
    for result in results:
        if result != "":
            # print(result)
            good_file_l.append(result)
    
    return good_file_l

In [91]:
getGoodFileParallelize(filelist, 297292, 160)


['/eos/purdue/store/group/local/hmm/nanoAODv6_private/FSRmyNanoProdData2017_NANOV4/SingleMuon/RunIISummer16MiniAODv3_FSRmyNanoProdData2017_NANOV4_un2017B-31Mar2018-v1/191007_094149/0000/myNanoProdData2017_NANO_281.root',
 '/eos/purdue/store/group/local/hmm/nanoAODv6_private/FSRmyNanoProdData2017_NANOV4/SingleMuon/RunIISummer16MiniAODv3_FSRmyNanoProdData2017_NANOV4_un2017B-31Mar2018-v1/191007_094149/0000/myNanoProdData2017_NANO_385.root',
 '/eos/purdue/store/group/local/hmm/nanoAODv6_private/FSRmyNanoProdData2017_NANOV4/SingleMuon/RunIISummer16MiniAODv3_FSRmyNanoProdData2017_NANOV4_un2017B-31Mar2018-v1/191007_094149/0000/myNanoProdData2017_NANO_546.root']

In [ ]:
# fname = "/eos/purdue/store/group/local/hmm/nanoAODv6_private/FSRmyNanoProdData2017_NANOV4/SingleMuon/RunIISummer16MiniAODv3_FSRmyNanoProdData2017_NANOV4_un2017B-31Mar2018-v1/191007_094149/0000/myNanoProdData2017_NANO_546.root"
# up_file = uproot.open(fname) 
# run_cond = up_file["Events"]['run'].array() == 297292
# run_cond = ak.sum(run_cond) > 0
# luminosityBlock_cond = up_file["Events"]['luminosityBlock'].array() == 160
# luminosityBlock_cond = ak.sum(luminosityBlock_cond) > 0

In [28]:
run_cond and luminosityBlock_cond

np.True_

In [29]:
ak.sum(up_file["Events"]['event'].array() ==296455078)

np.int64(1)

In [ ]:
import pandas as pd
"""
Load event dump df from Adish and compare
"""
Adish_df = pd.read_csv("EventDump_Adish_v2.csv")
df_unique_pairs = Adish_df[["run", "lumi"]].drop_duplicates()
df_unique_pairs = df_unique_pairs.astype("uint32")
df_unique_pairs

In [ ]:
import dask.dataframe as dd
import numpy as np
    

year = "2017"
label="Run2Rereco_synch_Apr23_2025"
load_path =f"/depot/cms/users/yun79/hmm/copperheadV1clean/{label}/stage1_output/{year}"

my_df = dd.read_parquet(f'{load_path}/data_B/*.parquet')
fields2compute = [
    "run",
    "luminosityBlock",
    "event",
    "mu1_pt",
    "mu1_eta",
    "mu1_phi",
    "mu2_pt",
    "mu2_eta",
    "mu2_phi",
    "dimuon_mass",
    "jet1_pt_nominal",
    "jet1_eta_nominal",
    "jet1_phi_nominal",
    "jet2_pt_nominal",
    "jet2_eta_nominal",
    "jet2_phi_nominal",
    "jj_mass_nominal",
    "nBtagMedium_nominal",
]
print(my_df.columns)
my_df = my_df[fields2compute]
# my_df = my_df[my_df["event"]==296503858].compute()
# my_df
my_df = my_df.dropna(subset=["mu1_pt"])

my_df = my_df.rename(columns={"luminosityBlock": "lumi"})
my_df = my_df.compute()

In [ ]:
# import pandas as pd


# filtered_df_merge = my_df.merge(df_unique_pairs, on=["run", "lumi"])
# print(filtered_df_merge)


In [ ]:
# filtered_df_merge.compute()
def filter_df(df, lumi, run):
    lumi_filter = df["lumi"] == lumi
    run_filter = df["run"] == run
    return df[lumi_filter & run_filter]
lumi_target = 160
run_target = 297292
my_df = filter_df(my_df, lumi_target, run_target)
Adish_df = filter_df(Adish_df, lumi_target, run_target)


In [ ]:
# np.sum(my_df["event"] ==296455078)
my_df

In [ ]:
Adish_df

In [ ]:
import matplotlib.pyplot as plt

diff = my_df["mu1_pt"] - Adish_df["m1pt"]
diff = diff/my_df["mu1_pt"]
plt.hist(diff, bins=50, histtype='step', label='Data')
# Add labels and title
plt.xlabel('Value')
plt.ylabel('Entries')
plt.title('1D Histogram Example')
plt.legend()

# Show the plot
plt.show()

In [116]:
import pandas as pd

# Example: df_A has more rows than df_B
# We want rows in A where 'event' is NOT in B

surplus_rows = my_df[~my_df["event"].isin(Adish_df["event"])]
surplus_rows

,run,lumi,event,mu1_pt,mu1_eta,mu1_phi,mu2_pt,mu2_eta,mu2_phi,dimuon_mass,jet1_pt_nominal,jet1_eta_nominal,jet1_phi_nominal,jet2_pt_nominal,jet2_eta_nominal,jet2_phi_nominal,jj_mass_nominal,nBtagMedium_nominal
entry,,,,,,,,,,,,,,,,,,


In [117]:
Adif_surplus_rows = Adish_df[~Adish_df["event"].isin(my_df["event"])]
Adif_surplus_rows 


,Unnamed: 0,Row,run,lumi,event,m1pt,m1eta,m1phi,m2pt,m2eta,...,j1pt,j1eta,j1phi,j2pt,j2eta,j2phi,mjj,nbjets,met,Unnamed: 21


In [93]:
import pandas as pd
import numpy as np

# Create example DataFrame
df = pd.DataFrame({
    "entry": ["A", "A", "A", "B", "B", "C", "D", "D", "D", "D"],
    "subentry": range(10),
    "value": np.random.rand(10)
})

print("Original DataFrame:")
print(df)

# --- Option 1: Keep first two subentries per entry ---
limited_df_first = df.groupby("entry", group_keys=False).head(2)

print("\nLimited to first 2 subentries per entry:")
print(limited_df_first)

# --- Option 2: Keep random two subentries per entry ---
limited_df_random = df.groupby("entry", group_keys=False).apply(lambda g: g.sample(n=min(2, len(g))))

print("\nLimited to random 2 subentries per entry:")
print(limited_df_random)


Original DataFrame:
  entry  subentry     value
0     A         0  0.625958
1     A         1  0.320118
2     A         2  0.172856
3     B         3  0.573381
4     B         4  0.572318
5     C         5  0.691362
6     D         6  0.024536
7     D         7  0.390214
8     D         8  0.069032
9     D         9  0.799289

Limited to first 2 subentries per entry:
  entry  subentry     value
0     A         0  0.625958
1     A         1  0.320118
3     B         3  0.573381
4     B         4  0.572318
5     C         5  0.691362
6     D         6  0.024536
7     D         7  0.390214

Limited to random 2 subentries per entry:
  entry  subentry     value
1     A         1  0.320118
0     A         0  0.625958
3     B         3  0.573381
4     B         4  0.572318
5     C         5  0.691362
6     D         6  0.024536
9     D         9  0.799289


/tmp/ipykernel_723707/1085525229.py:21: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  limited_df_random = df.groupby("entry", group_keys=False).apply(lambda g: g.sample(n=min(2, len(g))))
